In [24]:
from collections import *
from heapq import *
from functools import *
from math import *
import numpy as np
import copy
import time

data = None

with open('test.txt', 'r') as file:
    data = file.readlines()
    data = [d.strip() for d in data]
    data = [[cell for cell in row] for row in data]
#print(data)

In [26]:
from collections import Counter
from functools import cache

def getCell(data, x, y, n, m):
    if 0 <= x < n and 0 <= y < m:
        cell = data[y][x]
        if cell in 'SE':
            return '.'
        return cell
    return '#'

sys.setrecursionlimit(100000) 
def getCircle(x,y,rad):
    rad = rad-1
    arr = []
    for ny in range(-rad,rad+1):
        for nx in range(-rad,rad+1):
            dist = abs(ny)+abs(nx)
            if 1<=dist <=rad:
                arr.append((dist,nx+x,ny+y))
    return arr


def findSE(data):
    m, n = len(data), len(data[0])
    sx, sy, ex, ey = 0, 0, 0, 0
    for y, row in enumerate(data):
        for x, cell in enumerate(row):
            if cell == "S":
                sx, sy = x, y
            if cell == "E":
                ex, ey = x, y
    return sx, sy, ex, ey

def solve_maze(data, maxdepth=2):
    m, n = len(data), len(data[0])
    sx, sy, ex, ey = findSE(data)
    
    @cache
    def dfs(x, y, depth, px, py):
        # Base cases
        if not (0 <= x < n and 0 <= y < m):
            return float('inf')
        if (x, y) == (ex, ey):
            return 0
            
        cell = getCell(data, x, y, n, m)
        
        # Handle depth and wall constraints
        if cell == "#" or 0 < depth <maxdepth:
            depth -= 1
        if depth <= 0 or depth == maxdepth:
            if cell == '#':
                return float('inf')
        # Explore all directions
        directions = [(x, y-1), (x+1, y), (x, y+1), (x-1, y)]
        mn = float('inf')
        for nx, ny in directions:
            if (nx, ny) == (px, py):  # Avoid going back
                continue
            d = dfs(nx, ny, depth, x, y)
            if d != float('inf'):
                mn = min(mn, d + 1)
        return mn

    # Calculate shortest path without cheating
    nocheat = dfs(sx, sy, 0, -1, -1)
    
    # Calculate possible shortcuts
    arr = []
    for y in range(1, m-1):
        for x in range(1, n-1):
            if getCell(data, x, y, n, m) == ".":
                continue
                
            directions = [(x, y-1), (x+1, y), (x, y+1), (x-1, y)]
            nextTo = False
            dist = float("-inf")
            adjx,adjy = 0,0
            for nx, ny in directions:
                if getCell(data, nx, ny, n, m) == ".":
                    
                    nextTo = True
                    path_length = dfs(nx, ny, 0, -1, -1)
                    if path_length > dist:
                        dist = path_length
                        adjx,adjy = nx,ny
           # print(dist)
                        
            if nextTo:
                stos = nocheat - dist
                for nd,nx,ny in getCircle(x,y,maxdepth):
                    arr.append(nd+dfs(nx,ny,1,-1,-1)+stos+1)    
    # Count valid shortcuts
    filt = 50
    counter = Counter(arr)
    print(sorted([(d,f) for d, f in counter.items() if d + filt <= nocheat]))
    return sum(f for d, f in counter.items() if d + filt <= nocheat)
solve_maze(data,20)

[(8, 6), (10, 5), (12, 16), (14, 10), (16, 10), (18, 10), (20, 14), (22, 22), (24, 27), (26, 27), (28, 30), (30, 42), (32, 33), (34, 34)]


286

In [7]:
maxdepth = 2
def getCell(x,y):
    if 0<= x < n and 0 <= y < m:
        cell = data[y][x]
        if cell in 'SE':
            return '.'
        return cell
    return '#'
m,n = len(data),len(data[0])

def findSE():
    sx,sy,ex,ey = 0,0,0,0
    for y,row in enumerate(data):                                                                                                   
        for x,cell in enumerate(row):
            if cell == "S":
                sx,sy = x,y
            if cell == "E":
                ex,ey = x,y
    return sx,sy,ex,ey
    
sx,sy,ex,ey = findSE()

@cache
def dfs(x,y,depth,px,py):
    if not (0<= x< n and 0 <= y < m):
        return float('inf')
    if (x,y) == (ex,ey):
        return 0
    #print(x,y)
    cell = getCell(x,y)
    if cell == "#" or 0 < depth <maxdepth:
        depth -= 1
    if depth <= 0 or depth == maxdepth:
        if cell == '#':
            return float('inf')
    directions = [(x,y-1),(x+1,y),(x,y+1),(x-1,y)]
    mn = float('inf')
    for nx,ny in directions:
        if (nx,ny) == (px,py):
            continue
        d = dfs(nx,ny,depth,x,y)+1
        mn = min(mn,d)
    return mn

import sys
sys.setrecursionlimit(10000) 
nocheat = dfs(sx,sy,0,-1,-1)

arr = []
for y in range(1,m-1):
    for x in range(1,n-1):
        if getCell(x,y) == ".":
            continue
        directions = [(x,y-1),(x+1,y),(x,y+1),(x-1,y)]
        nextTo = False
        dist = float("inf")
        for nx,ny in directions:
            if getCell(nx,ny) == ".":
                nextTo = True
                dist = min(dist,nocheat-dfs(nx,ny,0,-1,-1))
        if nextTo:
            arr.append(dfs(x,y,maxdepth,nx,ny)+dist+1)

filt = 100
counter = Counter(arr)
#print(sorted(list(counter.items())))
sm = sum([f for d,f in counter.items() if d+filt <=nocheat])
print(sm)





RecursionError: maximum recursion depth exceeded in comparison